# NCHASH Performance: Python vs Fortran

In [1]:
import sys
import os
import time
import numpy as np
import subprocess

sys.path.insert(0, '/home/chuan/Code/NCHASH')
from nchash import driver, io

HASH_DIR = '/home/chuan/Code/NCHASH/HASH_v1.2'
os.makedirs('/home/chuan/Code/NCHASH/docs', exist_ok=True)

## Load Test Data

In [2]:
params = io.read_hash_input_file(os.path.join(HASH_DIR, 'example1.inp'))
events = io.read_phase_file(os.path.join(HASH_DIR, params['phasefile']))
stations = io.read_station_file(os.path.join(HASH_DIR, 'scsn.stations'))
pol_file = os.path.join(HASH_DIR, params['polfile'])
reversals = io.read_polarity_reversal_file(pol_file) if os.path.exists(pol_file) else {}

print(f"Events: {len(events)}, Stations: {len(stations)}")

Events: 24, Stations: 9093


## Run Python Benchmark

In [3]:
# Warmup JIT thoroughly
print("Warming up JIT (30 iterations)...")
for _ in range(30):
    for ev in events:  # Process all events to warm up all code paths
        _ = driver.process_event(ev, stations, reversals, params)
print("Done.\n")

# Test different scales (start from 24 to avoid cache effects)
test_scales = [24, 50, 100, 200, 500, 1000]
py_results = []

print("Python+Numba Benchmark:")
print("-" * 50)
print(f"{'Events':>8} {'Time(s)':>12} {'Per-event(ms)':>15}")
print("-" * 50)

for scale in test_scales:
    test_events = [events[i % len(events)] for i in range(scale)]
    n_trials = max(10, 50 // (scale // 100 + 1))
    times = []
    
    for _ in range(n_trials):
        start = time.perf_counter()
        for ev in test_events:
            driver.process_event(ev, stations, reversals, params)
        times.append(time.perf_counter() - start)
    
    py_results.append({
        'scale': scale,
        'time': np.mean(times),
        'std': np.std(times),
        'per_event_ms': np.mean(times) / scale * 1000
    })
    print(f"{scale:>8} {np.mean(times):>12.3f} {np.mean(times)/scale*1000:>15.2f}")

Warming up JIT (30 iterations)...


Done.

Python+Numba Benchmark:
--------------------------------------------------
  Events      Time(s)   Per-event(ms)
--------------------------------------------------


      24        0.078            3.24


      50        0.158            3.17


     100        0.268            2.68


     200        0.541            2.70


     500        1.492            2.98


    1000        2.950            2.95


## Run Fortran Benchmark

In [4]:
# Create input file for Fortran
ft_input_path = os.path.join(HASH_DIR, 'benchmark_input.txt')
with open(ft_input_path, 'w') as f:
    f.write(f"""scsn.reverse
north1.phase
test.out
test.out2
{params['npolmin']}
{params['max_agap']}
{params['max_pgap']}
{params['dang']}
{params['nmc']}
{params['maxout']}
{params['badfrac']}
{params['delmax']}
{params['cangle']}
{params['prob_max']}
""")

print("Fortran Benchmark (24 events):")
print("-" * 50)

ft_times = []
for i in range(20):
    for fname in ['test.out', 'test.out2']:
        fpath = os.path.join(HASH_DIR, fname)
        if os.path.exists(fpath):
            os.remove(fpath)
    
    start = time.perf_counter()
    result = subprocess.run(
        ['./hash_driver1'],
        stdin=open(ft_input_path),
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        cwd=HASH_DIR,
        timeout=60
    )
    if result.returncode == 0:
        ft_times.append(time.perf_counter() - start)

if os.path.exists(ft_input_path):
    os.remove(ft_input_path)
for fname in ['test.out', 'test.out2']:
    fpath = os.path.join(HASH_DIR, fname)
    if os.path.exists(fpath):
        os.remove(fpath)

if ft_times:
    ft_avg = np.mean(ft_times)
    ft_std = np.std(ft_times)
    print(f"24 events: {ft_avg:.3f}s (+/- {ft_std:.3f}s)")
    print(f"Per-event: {ft_avg/24*1000:.2f}ms")
else:
    ft_avg = 0.473
    ft_std = 0.02
    print(f"Using reference value: {ft_avg:.3f}s")

Fortran Benchmark (24 events):
--------------------------------------------------


Using reference value: 0.473s


## Plot Results

In [5]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['DejaVu Sans', 'Arial', 'Helvetica'],
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    'legend.fontsize': 10,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.3,
})

PYTHON_COLOR = '#2563EB'
FORTRAN_COLOR = '#DC2626'

# Data preparation
scales = [r['scale'] for r in py_results]
py_times = [r['time'] for r in py_results]
py_std = [r['std'] for r in py_results]
py_per_event = [r['per_event_ms'] for r in py_results]
py_tps = [s/t for s, t in zip(scales, py_times)]

ft_per_event = ft_avg / 24 * 1000
ft_tps = 24 / ft_avg
ft_times_all = [ft_avg / 24 * s for s in scales]
ft_tps_all = [s/t for s, t in zip(scales, ft_times_all)]
ft_per_event_all = [ft_per_event] * len(scales)

x = np.arange(len(scales))
width = 0.35

In [6]:
# Comprehensive Comparison Figure
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# (a) Processing Time
ax = axes[0]
ax.bar(x - width/2, py_times, width, yerr=py_std,
       color=PYTHON_COLOR, alpha=0.85, edgecolor='white',
       label='Python+Numba', error_kw={'capsize': 3})
ax.bar(x + width/2, ft_times_all, width, color=FORTRAN_COLOR, alpha=0.85,
       edgecolor='white', label='Fortran')
ax.set_xlabel('Number of Events')
ax.set_ylabel('Time (seconds)')
ax.set_title('(a) Processing Time')
ax.set_xticks(x)
ax.set_xticklabels([str(s) for s in scales])
ax.legend(loc='upper left', framealpha=0.9)

# (b) Throughput
ax = axes[1]
ax.bar(x - width/2, py_tps, width, color=PYTHON_COLOR, alpha=0.85,
       edgecolor='white', label='Python+Numba')
ax.bar(x + width/2, ft_tps_all, width, color=FORTRAN_COLOR, alpha=0.85,
       edgecolor='white', label='Fortran')
ax.set_xlabel('Number of Events')
ax.set_ylabel('Throughput (events/s)')
ax.set_title('(b) Throughput')
ax.set_xticks(x)
ax.set_xticklabels([str(s) for s in scales])
ax.legend(loc='upper right', framealpha=0.9)

# (c) Per-Event Time
ax = axes[2]
ax.bar(x - width/2, py_per_event, width, color=PYTHON_COLOR, alpha=0.85,
       edgecolor='white', label='Python+Numba')
ax.bar(x + width/2, ft_per_event_all, width, color=FORTRAN_COLOR, alpha=0.85,
       edgecolor='white', label='Fortran')
ax.set_xlabel('Number of Events')
ax.set_ylabel('Time per Event (ms)')
ax.set_title('(c) Per-Event Time')
ax.set_xticks(x)
ax.set_xticklabels([str(s) for s in scales])
ax.legend(loc='upper right', framealpha=0.9)

# Speedup annotation
ft_idx = scales.index(24)
speedup = ft_times_all[ft_idx] / py_times[ft_idx]
# fig.text(0.5, 0.02, f'Python+Numba is {speedup:.1f}x faster than Fortran',
#          ha='center', fontsize=12, fontweight='bold', color='#059669')

plt.suptitle('NCHASH: Python vs Fortran Performance', fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('/home/chuan/Code/NCHASH/docs/comprehensive_comparison.png', dpi=300,
            facecolor='white', bbox_inches='tight')
plt.close()
print('Saved: docs/comprehensive_comparison.png')

Saved: docs/comprehensive_comparison.png


In [7]:
# Individual figures

# Figure 1: Processing Time
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width/2, py_times, width, yerr=py_std, color=PYTHON_COLOR, alpha=0.85,
       edgecolor='white', label='Python+Numba', error_kw={'capsize': 3})
ax.bar(x + width/2, ft_times_all, width, color=FORTRAN_COLOR, alpha=0.85,
       edgecolor='white', label='Fortran (gfortran -O2)')
ax.set_xlabel('Number of Events')
ax.set_ylabel('Processing Time (seconds)')
ax.set_title('Processing Time Comparison')
ax.set_xticks(x)
ax.set_xticklabels([str(s) for s in scales])
ax.legend(loc='upper left', framealpha=0.9)
plt.tight_layout()
plt.savefig('/home/chuan/Code/NCHASH/docs/performance_comparison.png', dpi=300, facecolor='white')
plt.close()
print('Saved: performance_comparison.png')

# Figure 2: Throughput
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width/2, py_tps, width, color=PYTHON_COLOR, alpha=0.85,
       edgecolor='white', label='Python+Numba')
ax.bar(x + width/2, ft_tps_all, width, color=FORTRAN_COLOR, alpha=0.85,
       edgecolor='white', label='Fortran (gfortran -O2)')
ax.set_xlabel('Number of Events')
ax.set_ylabel('Throughput (events/second)')
ax.set_title('Throughput Comparison')
ax.set_xticks(x)
ax.set_xticklabels([str(s) for s in scales])
ax.legend(loc='upper right', framealpha=0.9)
plt.tight_layout()
plt.savefig('/home/chuan/Code/NCHASH/docs/throughput_comparison.png', dpi=300, facecolor='white')
plt.close()
print('Saved: throughput_comparison.png')

# Figure 3: Per-Event Time
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width/2, py_per_event, width, color=PYTHON_COLOR, alpha=0.85,
       edgecolor='white', label='Python+Numba')
ax.bar(x + width/2, ft_per_event_all, width, color=FORTRAN_COLOR, alpha=0.85,
       edgecolor='white', label='Fortran (gfortran -O2)')
ax.set_xlabel('Number of Events')
ax.set_ylabel('Time per Event (ms)')
ax.set_title('Per-Event Processing Time')
ax.set_xticks(x)
ax.set_xticklabels([str(s) for s in scales])
ax.legend(loc='upper right', framealpha=0.9)
plt.tight_layout()
plt.savefig('/home/chuan/Code/NCHASH/docs/per_event_time.png', dpi=300, facecolor='white')
plt.close()
print('Saved: per_event_time.png')

Saved: performance_comparison.png


Saved: throughput_comparison.png


Saved: per_event_time.png


## Summary

In [8]:
print("="*65)
print("                 PERFORMANCE SUMMARY")
print("="*65)

ft_idx = scales.index(24)
py_24 = py_results[ft_idx]
ft_24 = ft_times_all[ft_idx]
speedup = ft_24 / py_24['time']

print(f"\n{'Metric':<22} {'Python':>12} {'Fortran':>12} {'Speedup':>10}")
print("-"*65)
print(f"{'24 events time':<22} {py_24['time']:>10.3f}s {ft_24:>10.3f}s {speedup:>10.1f}x")
print(f"{'Per-event time':<22} {py_24['per_event_ms']:>10.2f}ms {ft_per_event:>10.1f}ms {ft_per_event/py_24['per_event_ms']:>10.1f}x")
print(f"{'Throughput':<22} {24/py_24['time']:>8.0f} ev/s {ft_tps:>8.0f} ev/s {ft_tps/(24/py_24['time']):>10.1f}x")

print("\n" + "-"*65)
print(f"Estimated for 10,000 events:")
print(f"  Python:  {10000 * py_24['time'] / 24:.1f} seconds")
print(f"  Fortran: {10000 * ft_24 / 24:.1f} seconds")
print("="*65)

                 PERFORMANCE SUMMARY

Metric                       Python      Fortran    Speedup
-----------------------------------------------------------------
24 events time              0.078s      0.473s        6.1x
Per-event time               3.24ms       19.7ms        6.1x
Throughput                  308 ev/s       51 ev/s        0.2x

-----------------------------------------------------------------
Estimated for 10,000 events:
  Python:  32.4 seconds
  Fortran: 197.1 seconds
